# Multi-Agent System: Smart Job Application Tailoring Assistant
Developed using CrewAI and LangChain stack

In [ ]:
# Optional: Install necessary libraries (if running locally)
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
# Silence warnings for a cleaner output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Environment setup
import os
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [ ]:
# Importing required modules and tools
from crewai import Agent, Task, Crew
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

In [ ]:
# Define tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./cv.md')
semantic_search_resume = MDXSearchTool(mdx='./cv.md')

In [ ]:
# Define Agents
# Agent 1: Job Market Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Analyze job postings to extract required qualifications",
    tools=[scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "An expert in job market trends, responsible for extracting core requirements "
        "and key insights from technical job listings."
    )
)

# Agent 2: Personal Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Generate detailed personal profiles from candidate data",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Specialist in analyzing candidate backgrounds through resumes, GitHub profiles, "
        "and online content to build comprehensive profiles for job matching."
    )
)

# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Enhance resumes by aligning them with job requirements",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Focused on optimizing resumes to best represent a candidate's qualifications "
        "based on the job market and employer expectations."
    )
)

# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Generate tailored interview questions and discussion points",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Creates targeted interview questions to ensure the candidate can communicate "
        "their skills and experience confidently during interviews."
    )
)

In [ ]:
# Define Tasks
research_task = Task(
    description=(
        "Extract core technical and soft skills, qualifications, and responsibilities "
        "from the job posting URL provided ({job_posting_url})."
    ),
    expected_output=(
        "Structured list of job requirements: skills, qualifications, experiences."
    ),
    agent=researcher,
    async_execution=True
)

profile_task = Task(
    description=(
        "Use the GitHub URL ({github_url}) and provided personal write-up ({personal_writeup}) "
        "to create a detailed personal and professional profile."
    ),
    expected_output=(
        "Comprehensive profile summarizing the candidate's skills, projects, and professional tone."
    ),
    agent=profiler,
    async_execution=True
)

resume_strategy_task = Task(
    description=(
        "Refine the resume based on the extracted job requirements and the candidate profile. "
        "Update summary, experience, skills, and education sections accordingly. Do not invent any info."
    ),
    expected_output=(
        "A highly targeted resume matching the job description."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

interview_preparation_task = Task(
    description=(
        "Generate personalized interview questions and suggested talking points "
        "based on the final tailored resume and job posting analysis."
    ),
    expected_output=(
        "Document with potential questions and points the candidate should be prepared to address."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

In [ ]:
# Define the Crew
job_application_crew = Crew(
    agents=[researcher, profiler, resume_strategist, interview_preparer],
    tasks=[research_task, profile_task, resume_strategy_task, interview_preparation_task],
    verbose=True
)

In [ ]:
# Inputs for the job application assistant
job_application_inputs = {
    'job_posting_url': 'https://testgorilla-1680540968.teamtailor.com/jobs/5955695-senior-machine-learning-engineer?promotion=1480373-trackable-share-link-linkedin-recruiter_clear-a',
    'github_url': 'https://github.com/abdessamed122',
    'personal_writeup': """Abdessamed is a driven Computer Science graduate and AI Research Assistant 
with hands-on experience in building intelligent systems, including chatbots, 
RAG applications, and multi-agent architectures using CrewAI and LangChain. 
He is skilled in Python, JavaScript, React, and modern AI frameworks like 
TensorFlow and PyTorch. Abdessamed has contributed to academic research on 
student enrollment prediction in Algeria and developed practical AI-powered 
tools for document understanding and summarization. Passionate about innovation, 
he thrives in fast-paced environments and is well-suited for roles in AI 
engineering, research, or full-stack development."""
}

In [ ]:
# Run the system
result = job_application_crew.kickoff(inputs=job_application_inputs)

In [ ]:
# Display output files
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))
display(Markdown("./interview_materials.md"))